# 📓 The GenAI Revolution Cookbook

**Title:** Prompt Library: How to Build a System That 10x Your Productivity

**Description:** Build a prompt library to reuse winning ChatGPT prompts, cut time by 80%, strengthen AI workflows, and impress hiring managers.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## What Problem Are We Solving?

You've written a prompt that works once, then fails on edge cases. You tweak it, test again, and lose track of what changed. A teammate asks for the "latest version," and you paste from Slack. Three weeks later, outputs drift and no one knows why.

This happens because prompts are treated as disposable strings, leading to instruction drift and inconsistent output contracts.

You'll implement a minimal prompt schema with format contracts, variables, and delimiters, plus a lightweight eval loop to measure improvement.

## What's Actually Happening Under the Hood

When prompts lack structure, several failure modes emerge:

- **Instruction dilution.** System rules and user input mix in a single block, causing the model to treat constraints as suggestions rather than hard boundaries.
- **Context interference.** Without delimiters, the model cannot reliably distinguish between instructions, examples, and live input, leading to hallucinated or off-spec outputs.
- **Unconstrained output shape.** No format contract means the model chooses its own structure, breaking downstream parsers and automation.
- **No negative examples.** The model sees only success cases, so it doesn't learn what to avoid or how to handle malformed input.

The diagram below shows where ambiguity enters and how structure mitigates it:

```mermaid
graph LR
    A[System Instructions] --> B[Delimiters]
    B --> C[User Input]
    C --> D[Model Processing]
    D --> E[Format Contract]
    E --> F[Structured Output]
    F --> G[Validation]
    G -->|Pass| H[Downstream Use]
    G -->|Fail| I[Reject & Log]
```

## How to Fix It

### Design a Minimal Prompt Schema

A prompt schema defines the structure and contract for every prompt in your library. Each prompt should include:

- **Role.** A single-sentence description of the model's task (e.g., "You are a code reviewer").
- **Format contract.** The exact output shape (e.g., JSON with specific keys, markdown with headers, or plain text with line breaks).
- **Variables.** Placeholders for dynamic input (e.g., `{code_snippet}`, `{user_query}`).
- **Delimiters.** Clear markers separating system instructions from user input (e.g., triple backticks, XML tags, or `---`).

Store each prompt as a versioned file with metadata (name, version, model, last updated).

### Isolate System Instructions From User Input

Place system-level rules in a dedicated section, separate from user-provided content. This prevents the model from treating user input as additional instructions.

**Before:**

In [ ]:
Write a product spec for {feature_name}. Include goals, user stories, and success metrics. {user_notes}

**After:**

In [ ]:
Role: You are a product manager writing a PRD.
Format: Return JSON with keys: goals, user_stories, success_metrics.
---
Feature: {feature_name}
Notes: {user_notes}

### Use Variables and Delimiters

Variables make prompts reusable across inputs. Delimiters protect against injection and context bleed.

Wrap user input in triple backticks or XML tags:

In [ ]:
Analyze the following code:

{code_snippet}

In [ ]:
Return a JSON object with keys: issues, suggestions, severity.

This ensures the model treats `{code_snippet}` as data, not instructions.

### Enforce Structured Outputs

Specify the exact output format in the system section. For JSON, list required keys and types. For markdown, define headers and sections.

Example format contract:

In [ ]:
Return JSON:
{
  "summary": "string",
  "action_items": ["string"],
  "priority": "high | medium | low"
}

Validate the output with a JSON Schema validator. If validation fails, log the prompt version, input, and output for debugging.

### Add Success Criteria and Constraints

Include 2–3 concrete success criteria and 1–2 constraints to anchor the model's behavior.

Success criteria:
- Summary must be under 100 words.
- Action items must be specific and assignable.

Constraints:
- Do not fabricate data not present in the input.
- If a required field is missing, return `{"error": "missing_field"}`.

### Provide Positive and Negative Examples

Show the model both correct and incorrect outputs. Negative examples teach boundary conditions.

Positive example:

In [ ]:
Input: "Add user authentication"
Output: {"goals": ["Secure user data"], "user_stories": ["As a user, I want to log in securely"], "success_metrics": ["Zero unauthorized access incidents"]}

Negative example:

In [ ]:
Input: "Add user authentication"
Bad Output: {"goals": "Secure user data"}
Why: goals must be an array, not a string.

## Measure Impact With Baselines

Before adopting structure, capture a baseline. Run your current prompt on 10–20 representative inputs and record:

- Output adherence (does it match the expected format?).
- Factuality (does it fabricate information?).
- Latency (how long does it take?).

After applying structure, re-run the same inputs and compare. Track the delta in a simple spreadsheet or JSON log.

Minimal eval set guidelines:
- Include 3–5 typical cases.
- Add 2–3 edge cases (e.g., missing fields, ambiguous input).
- Map each case to a metric (adherence, factuality, latency).
- Re-run evals after every prompt version change.

## Key Takeaways

- **Separate system and user sections.** Prevents instruction dilution.
- **Use variables and delimiters.** Protects against injection and context bleed.
- **Specify a format contract.** Ensures consistent, parseable outputs.
- **Add success criteria and constraints.** Anchors model behavior.
- **Provide negative examples.** Teaches boundary conditions.
- **Measure with baselines.** Quantifies improvement and catches regressions.

### When to Use This Pattern

- Outputs drift across runs or users.
- Downstream automation requires a consistent shape.
- Reviewers need to audit prompt logic.
- You're scaling prompts across a team or product.

### Token Cost and Latency Considerations

Structured prompts add tokens (delimiters, format contracts, examples). Keep scaffolds concise:
- Limit examples to 1–2 per type (positive/negative).
- Compress success criteria to essential bullets.
- Use short delimiters (e.g., `---` instead of verbose XML).

Track token counts and latency in your eval logs. If costs spike, trim examples or move format contracts to function calling schemas.

### Reproducibility in CI/CD

Pin prompt version and model version in automated tests. Artifact prompts and outputs for each test run. Store them in version control or an object store.

Checklist:
- Tag each prompt file with a semantic version (e.g., `v1.2.0`).
- Log prompt name, version, model, input, and output in CI.
- Compare outputs against expected results using JSON diff or string match.
- Fail the build if adherence drops below a threshold.

### Multi-Model Strategy

If you support multiple models, maintain a shared core prompt and model-specific overrides. Branch only when:
- Token window differences require truncation.
- Function calling or tool use differs.
- Output formatting quirks emerge (e.g., one model ignores JSON instructions).

Store overrides in separate files (e.g., `prompt_v1_gpt4.txt`, `prompt_v1_claude.txt`) and reference the shared core.

### Logging for Debugging

Log prompt name, version, input, and output for every production call. When outputs fail, query logs by prompt version to identify regressions. Tie logs to your "Automate Capture From Chat Sessions" workflow for incident review.

### Safety and Compliance Redlines

For engineering tasks, define 2–3 concrete redlines:
- Do not fabricate API endpoints or function signatures.
- Validate PII mask status before returning user data.
- If a required field is missing, return a validation error, not a guess.

Include these as constraints in the system section.